https://ethereum.stackexchange.com/questions/3667/difference-between-call-callcode-and-delegatecall

In [1]:
import web3
from web3 import Web3, EthereumTesterProvider
import json
import random
import time
from solcx import compile_files,get_solc_version,get_installed_solc_versions, set_solc_version,compile_source
import os

uncomment if you need to install solc

In [2]:
# from solcx import install_solc
# install_solc('v0.5.8')

In [3]:
get_installed_solc_versions()

['v0.4.21', 'v0.4.25', 'v0.5.8', 'v0.6.10', 'v0.6.12', 'v0.6.9', 'v0.7.0']

In [4]:
set_solc_version('v0.4.25')

# Contracts

In [5]:
contracts = '''

contract Caller {
  uint public n;
  address public sender;

  function callSetN(address _e, uint _n) {
    _e.call(bytes4(sha3("setN(uint256)")), _n); // Caller is not modified, Target's storage is set
  }

  function callcodeSetN(address _e, uint _n) {
    _e.callcode(bytes4(sha3("setN(uint256)")), _n); // Caller's storage is set, Target is not modified 
  }

  function delegatecallSetN(address _e, uint _n) {
    _e.delegatecall(bytes4(sha3("setN(uint256)")), _n); // Caller's storage is set, Target is not modified 
  }
}

contract Target {
  uint public n;
  address public sender;

  function setN(uint _n) {
    n = _n;
    sender = msg.sender;
  }
}

'''

# Connect to Eth

In [6]:
w3 = Web3(EthereumTesterProvider())
print("Connection:",w3.isConnected())
manager = w3.eth.accounts[0]
w3.eth.defaultAccount = manager

Connection: True


# Compile Contracts

In [7]:
compiled = compile_source(contracts)

# Deploy Caller Contract

In [8]:
tx_hash = w3.eth.contract(abi = compiled['<stdin>:Caller']['abi'],
                          bytecode = compiled['<stdin>:Caller']['bin']).constructor().transact()
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
caller_address = tx_receipt.contractAddress
print("caller_address",caller_address)
caller_contract = w3.eth.contract(address=caller_address,abi=compiled['<stdin>:Caller']['abi'])

caller_address 0xF2E246BB76DF876Cef8b38ae84130F4F55De395b


# Deploy Target Contract

In [9]:
tx_hash = w3.eth.contract(abi = compiled['<stdin>:Target']['abi'],
                          bytecode = compiled['<stdin>:Target']['bin']).constructor().transact()
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
target_address = tx_receipt.contractAddress
print("target_address",target_address)
target_contract = w3.eth.contract(address=target_address,abi=compiled['<stdin>:Target']['abi'])

target_address 0x2946259E0334f33A064106302415aD3391BeD384


# Call

In [10]:
tx_hash = caller_contract.functions.callSetN(target_address,50).transact({'from':w3.eth.accounts[1]})
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
print("gasUsed",tx_receipt.gasUsed)
print("account",w3.eth.accounts[2])
print("balance",w3.eth.getBalance(w3.eth.accounts[2]))
print("caller_contract.n",caller_contract.functions.n().call())
print("caller_contract.sender",caller_contract.functions.sender().call())
print("target_contract.n",target_contract.functions.n().call())
print("target_contract.sender",target_contract.functions.sender().call())

gasUsed 63899
account 0x6813Eb9362372EEF6200f3b1dbC3f819671cBA69
balance 1000000000000000000000000
caller_contract.n 0
caller_contract.sender 0x0000000000000000000000000000000000000000
target_contract.n 50
target_contract.sender 0xF2E246BB76DF876Cef8b38ae84130F4F55De395b


# Codecall

In [11]:
tx_hash = caller_contract.functions.callcodeSetN(target_address,20).transact({'from':w3.eth.accounts[2]})
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
print("gasUsed",tx_receipt.gasUsed)
print("account",w3.eth.accounts[2])
print("balance",w3.eth.getBalance(w3.eth.accounts[2]))
print("caller_contract.n",caller_contract.functions.n().call())
print("caller_contract.sender",caller_contract.functions.sender().call())
print("target_contract.n",target_contract.functions.n().call())
print("target_contract.sender",target_contract.functions.sender().call())

gasUsed 63811
account 0x6813Eb9362372EEF6200f3b1dbC3f819671cBA69
balance 999999999999999999936189
caller_contract.n 20
caller_contract.sender 0xF2E246BB76DF876Cef8b38ae84130F4F55De395b
target_contract.n 50
target_contract.sender 0xF2E246BB76DF876Cef8b38ae84130F4F55De395b


# Delegatecall

In [12]:
tx_hash = caller_contract.functions.delegatecallSetN(target_address,90).transact({'from':w3.eth.accounts[2]})
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
print("gasUsed",tx_receipt.gasUsed)
print("account",w3.eth.accounts[2])
print("balance",w3.eth.getBalance(w3.eth.accounts[2]))
print("caller_contract.n",caller_contract.functions.n().call())
print("caller_contract.sender",caller_contract.functions.sender().call())
print("target_contract.n",target_contract.functions.n().call())
print("target_contract.sender",target_contract.functions.sender().call())

gasUsed 33874
account 0x6813Eb9362372EEF6200f3b1dbC3f819671cBA69
balance 999999999999999999902315
caller_contract.n 90
caller_contract.sender 0x6813Eb9362372EEF6200f3b1dbC3f819671cBA69
target_contract.n 50
target_contract.sender 0xF2E246BB76DF876Cef8b38ae84130F4F55De395b
